In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for the testing dataset
mean = [0.1855, 0.1855, 0.1855]
std = [0.1813, 0.1813, 0.1813]

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

# Define your model architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 4)  # 4 classes
    
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)  # Flatten
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the model state dictionary
model_path = 'Model/Model'
state_dict = torch.load(model_path, map_location=device)

# Instantiate your model
model = CNNModel().to(device)

# Load state_dict into your model
model.load_state_dict(state_dict)
model.eval()  # Set the model to evaluation mode

# Path to the new image
image_path = 'glioma.jpeg'

# Load the new image
image = Image.open(image_path).convert('RGB')

# Apply the transformations to the image
image = test_transform(image)

# Add batch dimension and move to device
image = image.unsqueeze(0).to(device)

# Perform inference
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output.data, 1)

# Define class names
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Get the predicted label
predicted_labels = classes[predicted.item()]

print(f"Predicted Label: {predicted_labels}")


Predicted Label: meningioma
